In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import to_date, lit, date_format

try:
    spark.stop()
except:
    pass

spark = SparkSession.builder \
    .config("spark.driver.memory", "10g") \
    .config("spark.executor.memory", "10g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()


25/07/22 08:58:46 WARN Utils: Your hostname, DESKTOP-A7MMD62 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/22 08:58:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/22 08:58:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_application_test= spark.read.csv("../../data/raw/application_test.csv",
                               header=True,
                               inferSchema=True)

## Verificando quantidade de linhas e colunas no dataframe

In [3]:
df_application_test.createOrReplaceTempView("dados")
# Verificando a quantidade de linhas e colunas do DataFrame.

# Quantidade de linhas.
qtt_rows = df_application_test.count()

# Quantidade de colunas.
qtt_columns = len(df_application_test.columns)

# Quantidade de IDs únicos.
distinct_id_pos = spark.sql('''SELECT COUNT(DISTINCT `SK_ID_CURR`) as distinct_id_pos FROM dados ''')
distinct_id_pos.createOrReplaceTempView("distinct_id_pos")

# Imprimir o resultado.
print(f'Quantidade de linhas do DataFrame: {qtt_rows}')
print(f'Quantidade de colunas do DataFrame: {qtt_columns}')
distinct_id_pos.show()

25/07/22 08:58:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Quantidade de linhas do DataFrame: 92254
Quantidade de colunas do DataFrame: 171


+---------------+
|distinct_id_pos|
+---------------+
|          92254|
+---------------+



## Lendo os dados de Bureau

In [4]:
df_bureau= spark.read.parquet("../../data/books/bureau")

## Join da base de Application_Train com Bureau

In [5]:
app_test_full = df_application_test.join(df_bureau, "SK_ID_CURR", how="left")

In [6]:

print("Total de Colunas: " + str(len(app_test_full.columns)))

Total de Colunas: 325


## Lendo os dados de Previous Application

In [7]:
df_previous_app = spark.read.parquet("../../data/books/previous-app-fs")

## Join da base de Application_Train_Full com Previous Application

In [8]:
app_test_full = app_test_full.join(df_previous_app, "SK_ID_CURR", how="left")

In [9]:
print("Total de Colunas: " + str(len(app_test_full.columns)))
print("Total de linhas: " + str(app_test_full.count()))

Total de Colunas: 551


Total de linhas: 92254


## Salvando ABT em parquet

In [10]:
# Reparticionar para um único arquivo
abt_test = app_test_full.repartition(1)
abt_test.write.mode("overwrite").parquet('../../data/abt/abt_test')

In [11]:
spark.stop()